**Si ce n'est pas déjà fait, vous devez installer la librairie pyomo avant de pouvoir exécuter ce notebook.**

Pyomo est un langage de modélisation qui peut être utilisé en conjonction avec un certain nombre de solveurs. Pour plus d'informations sur Pyomo, vous pouvez également consulter la [documentation](http://www.pyomo.org/documentation).

# Modèle prescriptif de la détermination des prix - version détaillée

In [2]:
import pandas as pd

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    if "google.colab" in sys.modules:
        !pip install -q pyomo
    else:
        !conda install -c conda-forge pyomo
    assert(shutil.which("pyomo"))
    
if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass
    assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

import pyomo.environ as pe

# Blocs 1 & 2: données et paramètres d'entrée
Nous avons travaillé sur ce jeu de données au cours des dernières semaines. Toutefois, la liste de produits ci-dessous n'est qu'un sous-ensemble de ces produits; ces données correspondent aux données du fichier *predictedSales_Prob1.csv*. 

In [ ]:
url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/data/predictedSales_Prob1.csv'
predDemand = pd.read_csv(url)
predDemand

Dans ce notebook, nous aimerions travailler sur l'optimisation des prix des produits de cette liste. En particulier, chaque produit peut être vendu au prix de 2.5, 3.0 ou 3.5 dollars et nous souhaitons décider des prix auxquels ces produits sont proposés afin que notre revenu total soit maximisé. Comme nous l'avons appris avec l'analyse de la semaine dernière, le prix moyen des produits concurrents est un facteur affectant les ventes. Nous devons donc optimiser le prix de tous ces produits en parallèle pour atteindre l'optimum global.

Dans ce bloc, nous définissons aussi les listes d'index pour les produits (`N`) et pour les choix de prix (`M`).

In [ ]:
productList = ['1600027528', '1600027564', '3000006340', '3800031829']  # provient de la colonne UPC
priceList = [2.5, 3.0, 3.5]  # provient de la colonne PRICE
avgPriceValue = 3.0  # provient de la colonne AVG_PRICE_VALUE

N = list(range(len(productList)))  # [0, 1, 2, 3]
M = list(range(len(priceList)))  # [0, 1, 2]

# Bloc 3: création du modèle d'optimisation
Nous allons maintenant créer un modèle d'optimisation de détermination des prix, semblable à celui du cas Rue La La. Un modèle d'optimisation comprend (i) des variables de décision, (ii) une fonction objectif et (iii) des contraintes. Vous pouvez consulter le jupyter notebook *Modèles_production_avec_contraintes_linéaires.ipynb* pour un exemple simple de modèle d'optimisation utilisant Pyomo.

## Bloc 3.1: déclarations de variables
Nous utilisons `model.x` pour définir les variables de décision de notre modèle d'optimisation. Nous allons utiliser des variables binaires $x_{ij}$ qui prennent la valeur 0 ou 1 (non ou oui). En d'autres termes, $x_{ij} \in \{0,1\}$ où $i$ est l'index du produit et $j$ est l'index du prix. Nous pouvons définir formellement cette variable comme:
* $x_{ij}$ vaut 1 si le choix de prix $j$ est choisi pour le produit $i$, 0 sinon.

Veuillez noter qu'en Python (et dans de nombreux autres langages de programmation), l'index de départ est 0. La variable $x_{01}=1$ signifie que nous vendons le produit `'1600027528'` (produit à l'index 0 dans la liste de produits) au prix de 3,0\$ (prix à l'index 1 dans la liste de prix); voir la cellule de code précédente pour les listes de produits et de prix.

Dans ce bloc de code, nous créons un modèle (en utilisant la classe `pe.ConcreteModel()`) et déclarons les variables `x` (en utilisant `pe.Var(N, M, within=pe.Binary)`). La ligne `model.pprint()` affiche les détails du modèle que nous avons créé. Puisque nous n'avons créé que les variables, nous ne voyons ici que les variables, sans autres composantes. Vous pouvez visiter la [documentation](https://pyomo.readthedocs.io/en/stable/pyomo_overview/abstract_concrete.html) pour plus de détails sur la classe `ConcreteModel`.

In [ ]:
model = pe.ConcreteModel()  # création d'un modèle d'optimisation
model.x = pe.Var(N, M, within=pe.Binary)  # création des variables x
model.pprint()  # impression du modèle jusqu'à présent

## Bloc 3.2: ajout d'une fonction objectif
La forme générale de la fonction objectif est $\sum_{i=0}^3 \sum_{j=0}^2 p_{j} \cdot \tilde{D}_{ijk} \cdot x_{ij}$. 
Le premier terme de cette fonction objectif est $p_{j}$ qui correspond au $j$ème prix de la liste des prix. Par exemple, $p_{0}$ désigne le prix à l'index 0 de la liste de prix (2,5\\$ dans ce cas). 

Le deuxième terme de cette fonction objectif est $\tilde{D}_{ijk}$ qui correspond aux ventes prévues du produit $i$ lorsque ce produit est vendu au prix $j$ alors que le prix moyen de tous les produits concurrents, y compris le produit $i$, est égal à $k$. Dans notre modèle d'optimisation, $k$ est prédéfini à 3.0 (la colonne `'AVG_PRICE_VALUE'` de la première démonstration de la séance). Notez que $k$ correspond ici au prix moyen des produits concurrents et non pas à la somme des prix des produits concurrents comme dans le cas Rue La La; il est toutefois assez simple de passer de l'un à l'autre au besoin, il suffit de multiplier ou de diviser par le nombre de produits.

Finalement, le troisième et dernier terme correspond à $x_{ij}$ qui a été défini précédement.

En entrant le prix $p_{j}$ et le prix moyen $k$ dans le modèle prédictif entraîné, nous pouvons obtenir les ventes prévues correspondantes, soit $\tilde{D}_{ijk}$; c'est ce qui a été fait dans la première démonstration de cette séance (voir la colonne `'PRED_SALES'`). Nous utilisons `pe.Objective()` pour définir la fonction objectif et `sense=pe.maximize` pour indiquer que l'objectif correspond à une maximisation.

In [ ]:
# définition de la fonction objectif
# les chiffres de la demande proviennent de 'predictedSales_Prob1.csv'
model.obj = pe.Objective(
    sense=pe.maximize, 
    expr=2.5 * 94.9 * model.x[0,0] + 3.0 * 67.0 * model.x[0,1] + 3.5 * 46.4 * model.x[0,2] +
         2.5 * 24.1 * model.x[1,0] + 3.0 * 22.6 * model.x[1,1] + 3.5 * 19.8 * model.x[1,2] +
         2.5 *  6.2 * model.x[2,0] + 3.0 *  4.0 * model.x[2,1] + 3.5 *  3.0 * model.x[2,2] +
         2.5 * 32.9 * model.x[3,0] + 3.0 * 24.3 * model.x[3,1] + 3.5 * 20.4 * model.x[3,2])

## Bloc 3.3: ajout des contraintes
### Contraintes 1: un choix de prix doit être choisi pour chaque produit

En ce qui concerne le premier ensemble de contraintes, nous souhaitons nous assurer que chaque produit est vendu à un seul prix. Par conséquent, la forme générale de cet ensemble de contraintes est $\sum_{j=0}^2 x_{ij} = 1, \forall i\in\{0,1,2,3\}$. Nous utilisons `pe.Constraint()` pour définir les contraintes.

In [ ]:
# Contraintes #1
model.priceChoiceUPC0 = pe.Constraint(expr=model.x[0,0] + model.x[0,1] + model.x[0,2] == 1)
model.priceChoiceUPC1 = pe.Constraint(expr=model.x[1,0] + model.x[1,1] + model.x[1,2] == 1)
model.priceChoiceUPC2 = pe.Constraint(expr=model.x[2,0] + model.x[2,1] + model.x[2,2] == 1)
model.priceChoiceUPC3 = pe.Constraint(expr=model.x[3,0] + model.x[3,1] + model.x[3,2] == 1)

### Contrainte 2: la *moyenne* des prix de tous les produits concurrents doit être égale à $𝑘$

La deuxième contrainte garantit que le prix moyen de tous les **4** produits considérés dans notre modèle d'optimisation est égal au prix moyen prédéfini, qui est $k=3.0\$$. La forme générale est

$ \frac{ \sum_{i=0}^3 \sum_{j=0}^2 p_{j} \cdot x_{ij} }{4} =k \iff \sum_{i=0}^3 \sum_{j=0}^2 p_{j} \cdot x_{ij} = k\cdot 4$

Cela peut être fait en utilisant le bloc suivant:

In [ ]:
# Contrainte #2
model.sumPrice = pe.Constraint(
    expr=2.5 * model.x[0,0] + 3.0 * model.x[0,1] + 3.5 * model.x[0,2] +
         2.5 * model.x[1,0] + 3.0 * model.x[1,1] + 3.5 * model.x[1,2] + 
         2.5 * model.x[2,0] + 3.0 * model.x[2,1] + 3.5 * model.x[2,2] + 
         2.5 * model.x[3,0] + 3.0 * model.x[3,1] + 3.5 * model.x[3,2] == avgPriceValue * 4)

Nous pouvons maintenant afficher à nouveau le modèle pour voir toutes les composantes qui ont été créées.

In [ ]:
model.pprint()

# Bloc 4: solution et interprétation
Enfin, nous appelons le solveur et obtenons la solution. La première partie indique quel solveur nous voulons utiliser et la deuxième partie résout le modèle.

In [ ]:
# Résolution du modèle
pe.SolverFactory('glpk').solve(model) 

Vous pouvez maintenant afficher la solution en utilisant la méthode `model.display()` ci-dessous (`model.pprint()` affiche les décisions optimales, mais n'affiche pas la valeur de la solution optimale). Nous pouvons voir que $x_{00} = 1, x_{12} = 1, x_{22} = 1, x_{30} = 1$ (colonne Value) et la valeur optimale de l'objectif est $399.3\$$ (encore Value). En d'autres termes, nous atteignons le revenu optimal de $399.3\$$ lorsque:
- le produit '1600027528' (le produit à l'index 0) est vendu au prix de $2.5\$ $ (le prix à l'index 0), 
- les produits '1600027564' et '3000006340' (les produits aux index 1 et 2) sont vendus au prix de $3.5\$ $ (le prix à l'index 2), et
- le produit '3800031829' (le produit à l'index 3) est vendu au prix de $2.5\$ $ (le prix à l'index 0). 

Nous pouvons aussi facilement vérifier que toutes les contraintes sont satisfaites dans cet affichage (Body vs Lower/Upper).

In [ ]:
model.display()

Au besoin, il est aussi possible d'extraire la solution pour la stocker dans une autre structure de donnée. Par exemple, ci-dessous, on extrait les valeurs des variables `x` pour les stocker dans le dictionnaire `solution`.

In [ ]:
solution = {}
for i in N:
    for j in M:
        solution[i,j] = model.x[i,j].value  # ou pe.value(model.x[i,j])
solution

Par la suite, on extrait aussi aussi la valeur optimale de la fonction objectif et on la stock dans la variable `obj`.

In [ ]:
obj = pe.value(model.obj)
obj